In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import random
from matplotlib import pyplot as plt
import scipy.ndimage as nd
from tqdm.notebook import trange

%matplotlib inline

<div class="alert alert-success">
    <h1>Problema 1: Implementació de l'algorisme Seam Carving</h1>
    <p>
        L'algorisme Seam Carving ens permet reduir la mida de les imatges en una de les seves dues dimensions (horitzontalment, per exemple) a través d'un algorisme senzill de programació dinàmica. Ens permet fer la següent reducció:
    </p>    
    <img src='img/seamcarving.jpg' width='80%'><br>
    <b>Sembla màgia, no?</b><br><br>
    S'agafa la imatge original i se'n calcula el gradient. A continuació es generen tots els camins a través de la programació dinàmica, es troba un camí mínim i s'elimina. Es repeteix aquest procediment fins a obtenir el resultat desitjat. Observa'n un altre exemple:
    <img src='img/seamcarving_alg.png' width='80%'><br>
    En aquest cas hem reduit la imatge original en 150 píxels, horitzontalment. Com veieu, la major part dels objectes es conserven ja que s'han eliminat zones 'suaus' com el cel i la sorra.
    <br><br>
    <h3>Implementació</h3>
    <ol>
        <li> Implementeu una funció <code>superior_neighbors</code> que retorni els veïns superiors d'un punt d'una imatge 2-D. A la capçalera de la funció trobareu els tres casos que cal tenir en compte
        <li> Implementeu una funció <code>minimal_paths</code> que, usant el següent algorisme de programació dinàmica, construeixi la matriu de camins, $D$. Aquesta funció ha d'usar la funció <code>superior_neighbors</code> per obtenir els veïns.<br>
            Sigui $G$ la imatge de gradient i sigui $D$ la matriu de camins mínims que hem d'omplir, ambdues de dimensió $H \times W$ :
            $$D_{i,j} = \begin{cases}
                            G_{i,j}+\min\Big(D_{i-1, j},\ D_{i-1,j+1}\Big) \quad \text{if} \ j=0\\
                            G_{i,j}+\min\Big(D_{i-1, j-1},\ D_{i-1, j}\Big) \quad \text{if} \ j=W-1\\
                            G_{i,j}+\min\Big(D_{i-1, j-1},\ D_{i-1, j},\ D_{i-1,j+1}\Big) \quad \text{else}                                      
                        \end{cases}$$
        <li> Implementeu una funció <code>find_min_path</code> que retorni el camí mínim sobre la imatge anterior, $D$. Per trobar-lo, l'algorisme ha de començar per la part inferior de la imatge i anar pujant fins a arribar a la part superior.
        <li> Implementeu una funció <code>delete_path</code> que elimini el camí mínim retornat per la funció anterior de la imatge original.
        <li> Implementeu una funció <code>reduce_image</code> que repeteixi tot el procés anterior un nombre finit de vegades, $N$.
    </ol>
</div>

Funcions auxiliars:

In [11]:
def get_gradient(im):
    """
    Donada una imatge, en calcula el gradient
    
    Params
    ======
    :im: Imatge de la que en volem calcular el gradient
    
    Returns
    =======
    :gradient: Gradient horitzontal de la imatge
    
    """
    
    # Transformem la imatge a un sol canal (blanc i negre)
    im_blackwhite = np.dot(im[...,:3], [0.299, 0.587, 0.114])
    
    # Calculem el gradient usant sobel
    gradient = np.abs(nd.sobel(im_blackwhite))
    return gradient


def show_row(im_and_titles):
    """
    Donat un conjunt d'imatges i els seus títols, els mostra en una sola línia
    
    Params
    ======
    :im_and_titles: Llista de tuples en el format [(im, title), ...]    
    """
    
    # Creem una figura d'una sola línia
    fig, axs = plt.subplots(1,len(im_and_titles),figsize=(len(im_and_titles)*8,5))
    
    # Iterem el conjunt d'imatges i mostrem la imatge amb el seu títol
    for i, (im, title) in enumerate(im_and_titles):
        axs[i].imshow(im)
        axs[i].set_title(title)
        axs[i].set_axis_off()

    plt.tight_layout()
    plt.show()
    
    
def add_min_path(im, path, color=[1,0,0]):
    """
    Donada una imatge i un camí, afegeix el camí en un color donat. Per defecte, vermell.
    
    Params
    ======
    :im: Imatge sense el camí
    :path: Camí que volem afegir
    :color: Color del camí
    
    Returns
    =======
    :im: Imatge amb el camí afegit
    """
    for i, j in path:
        im[i][j]=color
    return im

Exemple d'ús:

In [ ]:
# Carreguem una imatge
im = plt.imread('img/beach.jpg')/255

# Creem un camí aleatori
i = im.shape[0]-1
path = [(i, im.shape[1]//2)]
while i >= 0:
    path.append((i, min(max(0, path[-1][1] + random.randint(-1,1)),im.shape[1]-1)))
    i-=1

# Afegim el camí a una imatge diferent
im_path = im.copy()
im_path = add_min_path(im_path, path)

# Usem la funció show_row per mostrar les imatges amb els seus títols.
im_titles = [(im, 'Original'), (im_path, 'Random path')]
show_row(im_titles)

## Gradient horitzontal

El <b>gradient horitzontal</b> d'una imatge ens permet trobar els punts de màxim canvi <b>vertical</b> en una imatge. Dit d'una altra forma, ens ressalta els contorns verticals.<br><br>
Observeu com podem usar la funció <code>get_gradient()</code> que us donem implementada:

In [ ]:
im = plt.imread('img/beach.jpg')/255

fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Mostrem la imatge
axs[0].imshow(im)
axs[0].set_title('Original image')
axs[0].set_axis_off()

# Mostrem el gradient horitzontal de la imatge
gradient = get_gradient(im)
axs[1].imshow(gradient, cmap='gray')
axs[1].set_title('Horizontal gradient')
axs[1].set_axis_off()

plt.tight_layout()
plt.show()

In [31]:
def superior_neighbors(mat, point):
    """
    Donada una matriu de mida H x W i punt, retorna els punts de la fila superior adjacents al punt passat com a paràmetre.
    Cal tenir en compte els següents casos. Considerant que el punt té coordenades (i,j):
        - Si el punt té coordenada j=0, vol dir que estem agafant un punt del marge esquerre de la imatge. Només s'han de retornar DOS veïns.
        - Si el punt té coordenada j=(W-1), vol dir que estem agafant un punt del marge dret de la imatge. Només s'han de retornar DOS veïns.
        - En la resta de casos, es retornen els tres veïns superiors.
        
    Params
    ======
    :mat: Una matriu 2-Dimensional
    :point: Un sol punt amb el format (i,j)
    
    Returns
    =======
    :neighbors: Una llista de dos o tres elements en funció de cada cas.
    """
    
    neighbors = np.array([])

    i, j = point

    # Sempre tindra un vei adalt seu
    neighbors = np.append(neighbors, mat[i - 1, j])
    # Si no es troba a la esquerra del tot de la imatge pot afegir el vei d'adalt esquerra
    if j > 0:
        neighbors = np.append(neighbors, mat[i - 1, j - 1])
    # Si no es troba a la dreta del tot de la imatge pot afegir el vei d'adalt dreta
    if j < mat.shape[1] - 1:
        neighbors = np.append(neighbors, mat[i - 1, j + 1])
    

    return neighbors


def minimal_paths(mat):
    """
    Creació de tots els camins mínims usant programació dinàmica.
    Cal usar la funció 'superior_neighbors' per trobar els veïns.
    
    Params
    ======
    :mat: Matriu 2-Dimensional d'entrada (gradient)
    
    Returns
    =======
    :ret: Matriu 2-Dimensional de la mateixa mida que 'mat' amb els camins mínims calculats.
    """
    
    ret = mat.copy()

    rows, cols = mat.shape

    # Apliquem la formula donada
    for i in range(1, rows):
        for j in range(cols):
            ret[i][j] = mat[i][j] + min(superior_neighbors(ret, (i,j)))
            
    return ret


def find_min_path(mat):
    """
    Donada una matriu, calcula el camí mínim sobre aquesta. L'algorisme ha de començar per la part inferior i buscar el següents punts.
    
    Params
    ======
    :mat: Matriu de camins mínims
    
    Returns
    =======
    :min_path: Una llista de tuples amb les coordenades (i,j) del camí mínim. La primera coordenada ha d'anar decrementant sempre en 1.
               Exemple. Suposant que una imatge té d'alçada 341 píxels, un possible camí seria: [(340, 120), (339, 121), (338,120), ..., (0, 151)] 
    """

    min_path = []

    rows = mat.shape[0]

    # Agafem la ultima fila i trobem l'index amb el minim gradient
    last_row = mat[-1,:]
    best = np.argmin(last_row)
    
    # Posem el node desde el que començarem a pujar
    current = (rows - 1, best)
    min_path.append(current)

    # Anem cap a adalt afegint els nodes veins amb minim gradient al path
    for _ in range(rows - 1, 0, -1):

        min_point = np.argmin(superior_neighbors(mat, current))
        
        # Segons com afegim els nodes a superior_neighbors(), trobem la coordenada del vei amb menys gradient
        if min_point == 0:
            offset = 0
        elif current[1] != 0 and min_point != 2:
            offset = -1
        else:
            offset = 1

        current = (current[0] - 1, current[1] + offset)
        min_path.append(current)
    
    return min_path



def delete_path(im, path):
    """
    Donat una imatge i un camí, elimina els pixels de la imatge que pertanyen del camí.
    Podeu usar la següent instrucció per inicialitzar la imatge. Això crea una imatge amb tots els valors a zero.
    
    im_new = np.zeros((im.shape[0], im.shape[1]-1, im.shape[2]))
    
    Params
    ======
    :im: Una imatge de mida H x W x 3
    :path: Un camí sobre la imatge. 
    
    Returns
    =======
    :im_new: Una nova imatge de mida H x (W-1) x 3 amb el camí eliminat
    """
    im_new = np.zeros((im.shape[0], im.shape[1]-1, im.shape[2]))

    # Invertim el path per esborrar de dalt a abaix
    j_dif = 0
    path.reverse()

    for i in range(im.shape[0]):
        # Trobem l'index de la cel·la que volem esborrar
        j_dif = path[i][1]
        for j in range(im.shape[1]):
            # Si ens trobem a l'esquerra d'on volem esborrar es copia la imatge i sino es copia movent-la a la esquerra una poisicio
            if j < j_dif:
                im_new[i, j] = im[i, j]
            elif j > j_dif:
                im_new[i, j - 1] = im[i, j]
    
    return im_new


def reduce_image(im, N=100):    
    """
    Implementació de l'algorisme Seam Carving. 
    Useu la funció 'show_row' al finalitzar per mostrar una figura amb tres subfigures:
        - Imatge original
        - Primer camí que s'elimina
        - Imatge resultant després de N iteracions
    
    Params
    ======
    :im: Imatge que volem reduir
    :N: Nombre de cops que repetirem l'algorisme
    """
    imatge = im.copy()
    imatg = im.copy()

    grad = get_gradient(imatge)
    imatg = add_min_path(imatg, find_min_path(minimal_paths(grad)))

    # Esborrem el minim path tants cops com ens demanin
    for _ in range(N):
        grad = get_gradient(imatge)
        imatge = delete_path(imatge, find_min_path(minimal_paths(grad)))

    show_row([(im, "Original"),[imatg,"Primer path"], (imatge, "Imatge final")])
    pass

Comproveu la vostra solució

In [ ]:
im = plt.imread('img/fireball.jpg')/255
reduce_image(im, 4)

In [ ]:
im = plt.imread('img/beach.jpg')/255
reduce_image(im, 150)

<div class="alert alert-success">
    <h1>Problema 2: Eliminar objectes d'una imatge</h1>
    <p>Modifiqueu l'algorisme anterior per a eliminar objectes d'una imatge. Només heu d'implementar la funció <code>remove_patch</code>. <br>Per exemple:</p>
    <img src='img/im_patched.png' width='80%'><br>
    <b>Pista:</b> Ens hem d'assegurar que els camins mínims sempre passin per l'interior del patch.
</div>

In [20]:
def add_patch(im, patch):
    """
    Donada una imatge i un patch. Mostra la imatge amb el patch d'un color donat. Per defecte, vermell.
    
    Params
    ======
    :im: La imatge a la que volem afegit el patch
    :patch: Patch amb quatre coordenades. Format: [(i1,j1), (i2, j2)]
    
    Returns
    =======
    :im: Imatge amb els píxels del patch en vermell.
    """
    
    for i in range(patch[0][0], patch[1][0]+1):
        for j in range(patch[0][1], patch[1][1]+1):
            im[i][j] = [1,0,0]
    return im

def inferior_neighbors(mat, point):
    """
    Donada una matriu de mida H x W i punt, retorna els punts de la fila inferior adjacents al punt passat com a paràmetre.
    Cal tenir en compte els següents casos. Considerant que el punt té coordenades (i,j):
        - Si el punt té coordenada j=0, vol dir que estem agafant un punt del marge esquerre de la imatge. Només s'han de retornar DOS veïns.
        - Si el punt té coordenada j=(W-1), vol dir que estem agafant un punt del marge dret de la imatge. Només s'han de retornar DOS veïns.
        - En la resta de casos, es retornen els tres veïns superiors.
        
    Params
    ======
    :mat: Una matriu 2-Dimensional
    :point: Un sol punt amb el format (i,j)
    
    Returns
    =======
    :neighbors: Una llista de dos o tres elements en funció de cada cas.
    """
    
    neighbors = np.array([])

    i, j = point

    # Sempre tindra un vei abaix seu
    neighbors = np.append(neighbors, mat[i + 1, j])
    # Si no es troba a la esquerra del tot de la imatge pot afegir el vei d'abaix esquerra
    if j > 0:
        neighbors = np.append(neighbors, mat[i + 1, j - 1])
    # Si no es troba a la dreta del tot de la imatge pot afegir el vei d'abaix dreta
    if j < mat.shape[1] - 1:
        neighbors = np.append(neighbors, mat[i + 1, j + 1])

    return neighbors


def find_min_path_with_patch(mat, patch):
    """
    Donada una matriu, calcula el camí mínim sobre aquesta passant pel patch.
    
    Params
    ======
    :mat: Matriu de camins mínims
    
    Returns
    =======
    :min_path: Una llista de tuples amb les coordenades (i,j) del camí mínim. La primera coordenada ha d'anar decrementant sempre en 1.
               Exemple. Suposant que una imatge té d'alçada 341 píxels, un possible camí seria: [(340, 120), (339, 121), (338,120), ..., (0, 151)] 
    """

    x1, y1 = patch[0] # La coordenada de més adalt i a la esquerra del pacth
    x2, y2 = patch[1] # La coordenada de més abaix i a la dreta del pacth

    min_idx = 0
    min_path = []

    rows, cols = mat.shape

    izq = -1
    der = 1

    # Si ens trobem en el cas que el patch esta a la esquerra o dreta del tot de la imatge no podem modificar la coordenada en dita direccio
    if y1 == 0:
        izq = 0
    if y2 == cols - 1:
        der = 0

    # Si no ens trobem en el cas que el patch esta abaix del tot hem de trobar el cami minim que es troba sota el patch
    if x2 != rows - 1: 

        # Trobem l'index amb menor gradient de la fila de just abaix del patch
        low_row = mat[x2 + 1, y1 + izq:y2 + 1 + der]
        min_idx = np.argmin(low_row) + y1 + izq
 
        # Posem el node desde el que començarem a baixar
        current = (x2+1, min_idx)
        min_path.append(current)

        # Anem cap a abaix desde el patch afegint els nodes veins amb minim gradient al path
        for _ in range(x2 + 1 + 1, rows):

            min_point = np.argmin(inferior_neighbors(mat, current))
            
            # Segons com afegim els nodes a superior_neighbors(), trobem la coordenada del vei amb menys gradient
            if min_point == 0:
                offset = 0
            elif current[1] != 0 and min_point != 2:
                offset = -1
            else:
                offset = 1

            current = (current[0] + 1, current[1] + offset)
            min_path.append(current)

        # Invertim la llista per tal d'anar d'abaix cap a adalt per la manera que esborrem el path a delete_path()
        min_path.reverse()


    # Afegim la columna esquerra del patch per tal d'esborrar-la (podriem posar qualsevol columna del path)
    for i in range(x2, x1 - 1, -1):
        min_path.append((i, y1))


    # Si no ens trobem en el cas que el patch esta adalt del tot hem de trobar el cami minim que es troba adalt el patch
    if x1 != 0:

        # Trobem l'index amb menor gradient de la fila de just adalt del patch
        high_row = mat[x1 - 1, y1 + izq: y2 + 1 + der]
        min_idx = np.argmin(high_row) + y1 + izq

        # Posem el node desde el que començarem a pujar
        current = (x1 - 1, min_idx)
        min_path.append(current)

        # Anem cap a adalt desde el patch afegint els nodes veins amb minim gradient al path
        for _ in range(x1 - 1, 0, -1):

            min_point = np.argmin(superior_neighbors(mat, current))
            
            # Segons com afegim els nodes a superior_neighbors(), trobem la coordenada del vei amb menys gradient
            if min_point == 0:
                offset = 0
            elif current[1] != 0 and min_point != 2:
                offset = -1
            else:
                offset = 1

            current = (current[0] - 1, current[1] + offset)
            min_path.append(current)

    return min_path

def remove_patch(im, patch):
    """
    Donada una imatge i un patch, n'elimina tots els punts interiors al patch.
    Useu la funció 'show_row' al finalitzar per mostrar una figura amb tres subfigures:
        - Imatge original
        - Imatge amb el patch de color vermell
        - Imatge resultant després d'eliminar el patch
    
    Params
    ======
    :im: Imatge original
    :patch: Patch amb dos parells de coordenades. 
            Format: [(i1,j1), (i2, j2)]. Sempre se satisfà que i1<i2, j1<j2.
            (i1, j1) és la coordenada superior esquerra del patch
            (i2, j2) és la coordenada inferior dreta del patch.
            Aquestes dues parelles s'han d'incloure com a part del patch.
    """

    y1 = patch[0][1] # La coordenada de més a la esquerra del pacth
    y2 = patch[1][1] # La coordenada de més a la dreta del pacth

    imatge = im.copy()
    imatge = add_patch(imatge, patch)
    im_patch = imatge.copy()

    # Esborrem tants camins minims com l'amplitud del patch
    for _ in range(y1, y2 + 1):
        grad = get_gradient(imatge)
        imatge = delete_path(imatge, find_min_path_with_patch(minimal_paths(grad), patch))
        # Reduim l'amplitud del patch
        patch = [patch[0], (patch[1][0], patch[1][1] - 1)]


    show_row([(im, "Original"),[im_patch,"Amb patch"], (imatge, "Sense patch")])
    
    pass

Comproveu la vostra solució.

Si ens donen un patch tant ample com la imatge el codi no funcionara perque la imatge s'elimina per complet intentant retornar una imatge que ja no existeix.

Proves a les diferents posibles posicions del patch:

In [ ]:
im = plt.imread('img/fireball.jpg')/255

patch = [(0, 0), (10, 10)]

remove_patch(im, patch)

In [ ]:
im = plt.imread('img/beach.jpg')/255
patch = [(156,391), (189,403)]

remove_patch(im, patch)